In [10]:
# img-> barcode extract -> api call -> get details and ingredients -> applied preprocessing on ingredients ->
# e1xtract chemicals based on category mahrness detection using model

In [11]:
# pipline
# 1. detect barcode from img
# 2. after detecting get the barcode number
# 3. fetch details through api 
# 4. category and ingredients 
# 5. match category with csv file (food , cleaning , etc)
# 6. ml model integration

In [43]:
import ctypes
import cv2
from pyzbar import pyzbar
from pyzbar import zbar_library

# ✅ Step 1: DLL absolute path (your path)
dll_path = r"C:\Users\dhruv\AppData\Roaming\Python\Python311\site-packages\pyzbar\libzbar-64.dll"

# ✅ Step 2: Force pyzbar to use your DLL
zbar_library.load = lambda: ctypes.cdll.LoadLibrary(dll_path)

# ✅ Step 3: Load the image
image_path = "Barcode_img.jpg"  # Replace with your actual image filename
image = cv2.imread(image_path)

# ✅ Step 4: Barcode detection
if image is None:
    print(f"❌ Could not load image: {image_path}")
else:
    barcodes = pyzbar.decode(image)

    if not barcodes:
        print("⚠️ No barcodes found.")
    else:
        for barcode in barcodes:
            x, y, w, h = barcode.rect
            data = barcode.data.decode('utf-8')
            type_ = barcode.type

            print(f"✅ Detected {type_} barcode: {data}")

            # Draw rectangle and label
            cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.putText(image, f"{data} ({type_})", (x, y - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)

        # ✅ Step 5: Show image with annotations
        cv2.imshow("Barcode Detection", image)
        cv2.waitKey(0)
        cv2.destroyAllWindows()


✅ Detected EAN13 barcode: 0204202472287


In [44]:
data

'0204202472287'

In [45]:
dll_path = r"C:\Users\dhruv\AppData\Roaming\Python\Python311\site-packages\pyzbar\libzbar-64.dll"
try:
    ctypes.cdll.LoadLibrary(dll_path)
    print("✅ DLL loaded successfully.")
except OSError as e:
    print("❌ DLL load failed:", e)


✅ DLL loaded successfully.


In [46]:
import requests

def get_general_product_info(barcode):
    url = f"https://api.upcitemdb.com/prod/trial/lookup?upc={barcode}"
    
    response = requests.get(url)
    if response.status_code != 200:
        print("❌ Failed to fetch data from UPCitemdb API.")
        return None
    
    data = response.json()
    
    if not data.get('items'):
        print("❌ No product found for this barcode.")
        return None

    item = data['items'][0]
    title = item.get('title', 'N/A')
    brand = item.get('brand', 'N/A')
    category = item.get('category', 'N/A')
    description = item.get('description', 'N/A')

    print(f"✅ Product Title: {title}")
    print(f"Brand: {brand}")
    print(f"Category: {category}")
    print(f"Description: {description}")

    return {
        'title': title,
        'brand': brand,
        'category': category
    }

# Example usage:
# get_general_product_info("028400642511")
get_general_product_info("012993441012")  # Sample barcode
# get_general_product_info(data)

✅ Product Title: LaCroix Enhanced Sparkling Water Passionfruit - 12 fl oz
Brand: LaCroix
Category: Food, Beverages & Tobacco > Beverages > Water > Carbonated Water
Description: Passionfruit now available nationwide! Check your local retailers as our newest flavor is added to store shelves throughout early 2015 Gender: unisex.


{'title': 'LaCroix Enhanced Sparkling Water Passionfruit - 12 fl oz',
 'brand': 'LaCroix',
 'category': 'Food, Beverages & Tobacco > Beverages > Water > Carbonated Water'}

In [47]:
# Mapping category keywords to harm score columns
category_mapping = {
    'food': 'harm_score_food',
    'cosmetic': 'harm_score_cosmetic',
    'personal care': 'harm_score_personal_care',
    'cleaning': 'harm_score_cleaning',
    'stationery': 'harm_score_stationery',
    'household': 'harm_score_household',
    'medicine': 'harm_score_medicine',
    'others': 'harm_score_others'
}


In [48]:
api_category = "Food, Beverages & Tobacco > Food Items > Cooking & Baking Ingredients > Flour"

# Find matching key in mapping
matched_key = next((key for key in category_mapping if key in api_category.lower()), 'others')


harm_score_column = category_mapping[matched_key]

print("Using column:", harm_score_column)


Using column: harm_score_food


In [49]:
import pandas as pd

# Your chemical list from the detected product (example)
chemicals = ['Citric Acid', 'Xanthan Gum', 'Sugar']

# Your harm score dataframe
df = pd.read_csv("chemical_harmness_category.csv")

# Filter relevant chemicals
filtered_df = df[df['chemical_name'].isin(chemicals)]

# Sum the relevant harm score column
total_harm_score = filtered_df[harm_score_column].sum()

print(f"Total Harm Score ({matched_key}):", total_harm_score)


Total Harm Score (food): 20
